In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as optimize
import os
import sys
import collections
import itertools
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import sklearn.metrics
from scipy.stats import mode
from scipy.spatial.distance import squareform
from fastdtw import fastdtw
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
input_data = pd.read_csv('../Data/Input_features_1960.csv')
input_data

,Unnamed: 0,Month,Fedfund_rate,GDP_pch,CPI_pc1,Loan_pch,House_ch
0,0,1960-01-01,3.99,2.657590,1.24095,10.2,-197.0
1,1,1960-02-01,3.97,1.675410,1.41379,10.1,-164.0
2,2,1960-03-01,3.84,0.693230,1.51881,6.7,-511.0
3,3,1960-04-01,3.92,-0.288950,1.93237,6.6,-301.0
4,4,1960-05-01,3.85,0.086070,1.82507,3.3,-227.0
...,...,...,...,...,...,...,...
760,760,2023-05-01,5.06,1.310517,4.12884,6.0,40.0
761,761,2023-06-01,5.08,1.690863,3.09200,3.0,-143.0
762,762,2023-07-01,5.12,2.071210,3.29908,2.4,80.0
763,763,2023-08-01,5.33,2.071210,3.70750,6.3,-200.0


In [3]:
output_data = pd.read_csv('../Data/Employment_pch_1960.csv')
output_data = output_data[:-1]
output_data

,Unnamed: 0,Month,Total_private_pch,Construction_pch
0,0,1960-01-01,0.35148,-1.20994
1,1,1960-02-01,0.47860,1.05925
2,2,1960-03-01,-0.55860,-5.63380
3,3,1960-04-01,0.76205,4.26935
4,4,1960-05-01,-0.50995,0.56591
...,...,...,...,...
760,760,2023-05-01,0.19161,0.31642
761,761,2023-06-01,0.06450,0.36588
762,762,2023-07-01,0.10867,0.15085
763,763,2023-08-01,0.08535,0.37655


# Linear Regression

In [4]:
X = input_data.iloc[:,2:]
y = output_data.iloc[:,2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
model.score(X_test, y_test)

-0.09581145742076091

In [5]:
pred_sign = np.sign(pred)
y_test_sign = np.sign(y_test)
classification_accuracy = 1 - 0.5 * np.sum(np.abs(pred_sign - y_test_sign)) / np.size(y_test_sign)
classification_accuracy

0.8

In [6]:
pred_train = model.predict(X_train)
pred_train_sign = np.sign(pred_train)
y_train_sign = np.sign(y_train)
train_accuracy = 1 - 0.5*np.sum(np.abs(pred_train_sign - y_train_sign))/np.size(y_train_sign)
train_accuracy # train accuracy for classification via linear regression

0.7588785046728972

# Logistic Regression

In [7]:
X = input_data.iloc[:,2:]
y = output_data.iloc[:,2]
y = np.sign(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)  # test accuracy (boolean labels)

0.8391304347826087

# Decision Tree Classifier

In [8]:
X = input_data.iloc[:,2:]
y = output_data.iloc[:,2]
y = np.sign(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)  # test accuracy (boolean labels)

0.8130434782608695

# SVM Classifier

In [9]:
X = input_data.iloc[:,2:]
y = output_data.iloc[:,2]
y = np.sign(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = SVC()
model.fit(X_train, y_train)
model.score(X_test, y_test)  # test accuracy (boolean labels)

0.782608695652174

# KNN (with dynamic time warping)

In [10]:
from fastdtw import fastdtw
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def calculate_dtw(series1, series2):
    distance_dtw, _ = fastdtw(series1, series2)
    return distance_dtw

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

knn_model = KNeighborsClassifier(n_neighbors=4, metric=calculate_dtw)
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7565217391304347


# Naive Bayes

In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

y_labels = (y > 0).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.3, random_state=0)
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7782608695652173
Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.33      0.37        46
           1       0.84      0.89      0.87       184

    accuracy                           0.78       230
   macro avg       0.63      0.61      0.62       230
weighted avg       0.76      0.78      0.77       230


# Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier

y_sign = np.sign(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_sign, test_size=0.3, random_state=0)
rf_model = RandomForestClassifier(n_estimators=100, random_state=3)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8652173913043478
Classification Report:
              precision    recall  f1-score   support

        -1.0       0.76      0.48      0.59        46
         1.0       0.88      0.96      0.92       184

    accuracy                           0.87       230
   macro avg       0.82      0.72      0.75       230
weighted avg       0.86      0.87      0.85       230


# Baseline

In [13]:
X = input_data.iloc[:,2:]
y = output_data.iloc[:,2]

y_labels = (y > 0).astype(int)

# predict next month's class to be the same as this month's class
y_pred = y_labels.shift(1).dropna()

# split the data into training and testing sets along with corresp. indices
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y_labels, np.arange(len(y_labels)), test_size=0.3, random_state=0)

# check accuracy of baseline
accuracy = np.sum(y_pred[indices_test] == y_test) / np.size(y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8913043478260869
